In [2]:
import sys
!{sys.executable} -m pip install yfinance sec-parser

  Using cached yfinance-0.2.66-py2.py3-none-any.whl.metadata (6.0 kB)
  Using cached sec_parser-0.58.1-py3-none-any.whl.metadata (18 kB)
  Using cached sec_parser-0.58.1-py3-none-any.whl.metadata (18 kB)
  Using cached multitasking-0.0.12-py3-none-any.whl
  Using cached peewee-3.18.2-cp311-cp311-linux_x86_64.whl
  Using cached multitasking-0.0.12-py3-none-any.whl
  Using cached peewee-3.18.2-cp311-cp311-linux_x86_64.whl
  Using cached curl_cffi-0.13.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached curl_cffi-0.13.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached cssutils-2.11.1-py3-none-any.whl.metadata (8.7 kB)
  Using cached cssutils-2.11.1-py3-none-any.whl.metadata (8.7 kB)
  Using cached loguru-0.7.3-py3-none-any.whl.metadata (22 kB)
  Using cached loguru-0.7.3-py3-none-any.whl.metadata (22 kB)
  Using cached sec_downloader-0.11.2-py3-none-any.whl.metadata (10 kB)
  Using cached sec_downloader-0.11.2-

In [8]:
from typing import TypedDict


class MetricData(TypedDict):
    volatility: float
    variation: float


# Get the market cap, variation and volatility in % in the last week, month and year about a given stock
def get_stock_data(symbol: str) -> dict:
    import yfinance as yf
    import numpy as np
    from datetime import datetime, timedelta, timezone

    continent_map = {
        'United States': 'North America',
        'USA': 'North America',
        # Add more mappings as needed
    }

    stock = yf.Ticker(symbol)
    info = stock.info
    cap = info.get('marketCap')
    recommendation = info.get('recommendationKey')
    
    # Map recommendation to indicator out of 5
    recommendation_map = {
        'STRONG_BUY': 5,
        'BUY': 4,
        'HOLD': 3,
        'SELL': 2,
        'STRONG_SELL': 1
    }
    indicator = recommendation_map.get(recommendation.upper() if recommendation else None, None)
    
    results = {'cap': cap, 'yahoo_pespective': indicator}
    results['stock_exchange'] = info.get('exchange')
    results['country'] = info.get('country')
    results['sector'] = info.get('sector')
    results['industry'] = info.get('industry')
    results['continent'] = continent_map.get(results['country'], 'Unknown')
    
    end_date = datetime.now(timezone.utc)

    periods = {
        '1w': end_date - timedelta(weeks=1),
        '1m': end_date - timedelta(days=30),
        '1y': end_date - timedelta(days=365)
    }

    for period, start_date in periods.items():
        hist = stock.history(start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
        if len(hist) < 2:
            results[f'variation_{period}'] = None
            results[f'volatility_{period}'] = None
            continue

        returns = hist['Close'].pct_change().dropna()
        volatility = returns.std() * np.sqrt(252) * 100  # Annualized %
        variation = ((hist['Close'].iloc[-1] - hist['Close'].iloc[0]) / hist['Close'].iloc[0]) * 100

        results[f'variation_{period}'] = round(variation, 2)
        results[f'volatility_{period}'] = round(volatility, 2)
    results['variation'] = float(min(5, max(0, 0.2 * results['variation_1w'] + 0.3 * results['variation_1m'] + 0.5 * results['variation_1y'])))
    results['volatility'] = float(min(5, max(0, 5 - 0.2 * results['volatility_1w'] - 0.3 * results['volatility_1m'] - 0.5 * results['volatility_1y'])))

    return results

In [ ]:
# print(get_stock_data("AAPL"))

{'cap': 3995082424320, 'yahoo_pespective': 4, 'stock_exchange': 'NMS', 'country': 'United States', 'sector': 'Technology', 'industry': 'Consumer Electronics', 'continent': 'North America', 'variation_1w': np.float64(0.58), 'volatility_1w': np.float64(6.67), 'variation_1m': np.float64(5.33), 'volatility_1m': np.float64(25.04), 'variation_1y': np.float64(22.35), 'volatility_1y': np.float64(32.77), 'variation': 5.0, 'volatility': 0.0}
